In [ ]:
from renderer import Chat
chat = Chat.fromKey("914472")
doc = chat.build()
with open("test.md", "w") as f:
    f.writelines(doc.render())

In [ ]:
with open("foobar.json", "w") as f:
    from renderer import DB
    import json
    json.dump(DB.getDocument("chat-logs", "1096206"), f)

In [ ]:
class dummy:
    @staticmethod
    def foo():
        print("foo")

dummy.foo = lambda : print("bar")

In [ ]:
dummy.foo()

In [ ]:
from pathlib import PurePath

PurePath("/foo/bar") / "baz"